<a href="https://colab.research.google.com/github/agrimwood/medphys_helper/blob/main/MPC_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports for JupyterLite
%pip install -q ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import glob
from matplotlib import pyplot as plt


In [ ]:
import seaborn as sns
import itertools
# read all csv files to dataframe
lst = []
for f in glob.glob('/content/*.csv'):
  df = pd.read_csv(f)
  lst.append(df)
df = pd.concat(lst,ignore_index=True)
df.Energies = df.Energies.str.upper()

# simple interactive plot of MPC data
pd.set_option('copy_on_write', True)
@interact(TestParameter=df.columns[6::2],
          linac=[x for x in df['SN'].unique() if str(x) != 'nan'],
          energy=[x for x in df['Energies'].unique() if str(x) != 'nan'],
          auto_yaxis=True,
          ymin=df[df.columns[6::2]].min().min(),
          ymax=df[df.columns[6::2]].max().max(),
          dtmin=widgets.DatePicker(description="Start Date"),
          dtmax=widgets.DatePicker(description="End Date"))
def dataPlot(TestParameter,linac,energy,dtmin,dtmax,auto_yaxis,ymin,ymax):
    dfplt = df[(df['SN']==linac) & (df['Energies']==energy)]
    dfplt['Testdate'] = pd.to_datetime(dfplt['Testdate'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    dfplt[TestParameter] = pd.to_numeric(df[TestParameter], errors='coerce')
    dfplt = dfplt[['Testdate',TestParameter,TestParameter+" Result"]].dropna()
    fig, ax = plt.subplots(layout='constrained', figsize=(12.5,5))
    if auto_yaxis:
      p1 = sns.scatterplot(x='Testdate', y=TestParameter, data=dfplt, hue=TestParameter+" Result", ax=ax)
      p2 = sns.lineplot(x='Testdate', y=TestParameter, data=dfplt, size=0.5, legend=False, color = "black", ax=ax)
    else:
      p1 = sns.scatterplot(x='Testdate', y=TestParameter, data=dfplt, hue=TestParameter+" Result", ax=ax)
      p2 = sns.lineplot(x='Testdate', y=TestParameter, data=dfplt, size=0.5, legend=False, color = "black", ax=ax)
      plt.ylim([ymin,ymax])
    plt.xlim([dtmin,dtmax])
    plt.title(energy+": "+TestParameter)
    plt.ylabel(TestParameter)
    plt.xlabel("Date")
    plt.show()
    return